This notebook extracts information about time and location of spikes from the hdf5 file with the BrainGrid simulation data and converts it into a dataframe with the following columns: time_step (time step when the spike happened), xloc, yloc (coordinates of the spike). It then extracts spacial and temporal pre-burst and non-burst windows, labels them, and saves the result in a csv.

Author: Mariia Lundvall lundvm@uw.edu <br>
Date: 01/21/2019

In [1]:
import numpy as np
import pandas as pd
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    import h5py
from tqdm import tqdm_notebook as tqdm
import gc
import numpy.ma as ma

<b> NOTE: update paths in the cell below prior to running this notebook <b>

In [10]:
# location of the HDF5 file containing simulation data
hdf5 = '/home/NETID/lundvm/data/tR_1.0--fE_0.90.h5'
# location where to save the spike data extracted by create_spike_matrix
spikes_csv = '/home/NETID/lundvm/data/spikes.csv'
# location of the file containing bursts data 
# this file is created by the getAvalanches.m script
bursts_csv = '/home/NETID/lundvm/data/tR_1.0--fE_0.90/allAvalBurst.csv'
# location where to save extracted temporal training data
temp_path = '/home/NETID/lundvm/data/train_data/temp_'
# location where to save extracted spacial training data
space_path = '/home/NETID/lundvm/data/train_data/space_'

In [3]:
def create_spike_matrix(f):
    
    """
    Takes in an HDF5 file produced by the BrainGrid simuation, extracts information about time 
    and location of spikes and converts it into a dataframe with the following columns: time_step 
    (time step when the spike happened, type: int), xloc, yloc (coordinates of the spike, type: int).
    
    Args: 
        f(HDF5): loaded HDF5 file to read data from.
    
    Returns:
        spikes_loc(pandas dataframe): dataframe that contains spiking data. The dataframe has the 
            following columns: time_step (time step when the spike happened, type: int), xloc, yloc 
            (coordinates of the spike, type: int).
    """
    
    # get the spikes time steps and coordinates data from the hdf5 file and convert the data 
    # into numpy arrays
    spikes = np.array(f['/spikesProbedNeurons'], dtype='uint32')
    xloc = np.array(f['/xloc'], dtype='uint8')
    yloc = np.array(f['/yloc'], dtype='uint8')
    # m is the max number of spikes per neuron, n is the number of neurons in the simulation
    m, n = spikes.shape
    # transform the spikes matrix:
    # 1. Traspose so that each row is a sequence of spikes of one neuron (instead of a column)
    # 2. Flattem the matrix 
    # 3. Reshape into a 2d array from (m*n, ) to (m*n, 1). This is needed for further processing. 
    spikes = np.transpose(spikes).flatten().reshape(m*n, 1)
    # create a mask to remove non-spikes (where time step=0)
    mask = ma.masked_equal(spikes, 0).reshape(m*n, )
    # Transform the coordinate vectors:
    # 1. Make the vectors match the time step sequence. Repeat the values so that first m values 
    # in the x and y vectors are the coordinates of the first neuron
    # 2. Remove the coordinates corresponding to non-spikes
    # 3. Concatenate x and y, the result is an array xy of shape (m*n, 2)
    xloc = np.compress(mask, np.repeat(xloc, m).reshape(m*n, 1), axis=0)
    yloc = np.compress(mask, np.repeat(yloc, m).reshape(m*n, 1), axis=0)
    xy = np.concatenate((xloc, yloc), axis=1)
    # delete xloc and yloc to free memory
    del xloc
    del yloc
    gc.collect()
    # Remove non-spikes from the time step array, concatenate it to xy, 
    # and convert the result into a dataframe
    spikes_loc = pd.DataFrame(np.concatenate((np.compress(mask, spikes, axis=0), xy), axis=1))
    # delete spikes to free the memory
    del spikes
    gc.collect()
    spikes_loc.rename(columns={0:'time_step', 1:'xloc', 2:'yloc'}, inplace=True)
    spikes_loc.sort_values(by='time_step', inplace=True)
    spikes_loc.reset_index(drop=True, inplace=True)
    spikes_loc['time_step'] = spikes_loc['time_step'].astype('int32')
    spikes_loc['xloc'] = spikes_loc['xloc'].astype('uint8')
    spikes_loc['yloc'] = spikes_loc['yloc'].astype('uint8')
    
    return spikes_loc

In [4]:
def get_training_data(windows, gap, spikes, x, y, mask, n, temp_path, space_path):
    
    """
    Takes in the spiking data, extracts spacial and temporal pre-bursts and non-bursts 
    windows, adds a label (0 for non-burst, 1 for pre-burst) to each window, and saves 
    the windows as a csv (one csv per window size, separate csvs with temporal and spacial data).

    Args:
        windows(list of ints): the list containing window sizes
        gap(int): the size of the gap between the beginning of a burst and a non-burst window.
        spikes(numpy 1d array): array contaning spikes time steps
        x(numpy 1d array): array contaning x coordinates of the spikes
        y(numpy 1d array): array containing y coordinates of the spikes
        mask(boolean array): array of the same length as spikes, containing True for time steps 
            that indicate the starts of bursts
        n(int): number of identified bursts in the simulation

    Returns:
        none
        saves csv files with temporal and spacial training data
    """
    
    m = spikes.shape[0]
    for w in tqdm(windows):
        print('Extracting windows of size ' + str(w))
        # initialize pre-burst and non-burst arrays
        temp_preburst = np.ones((n, w+1))
        temp_nonburst = np.zeros((n, w+1))
        space_preburst = np.ones((n, w*2+1))
        space_nonburst = np.zeros((n, w*2+1))
        index = 0
        for i in range(1, m):
            if not mask[i-1] and mask[i]:
                # attempt to remove bad bursts. If the gap width is smaller than the
                # gap size, we must be catching the beginning of a burst, so don't extract those windows
                if spikes[i] - spikes[i-gap] >= gap:
                    # get the temporal windows
                    temp_preburst[index, 0:w] = spikes[i-w:i] - spikes[i-w]
                    temp_nonburst[index, 0:w] = spikes[i -
                                                       w-gap:i-gap] - spikes[i-w-gap]
                    # get the spacial windows
                    space_preburst[index, 0:w*2] = np.concatenate(
                        (x[i-w:i].reshape(w, 1), y[i-w:i].reshape(w, 1)), axis=1).flatten()
                    space_nonburst[index, 0:w*2] = np.concatenate((x[i-w-gap:i-gap].reshape(
                        w, 1), y[i-w-gap:i-gap].reshape(w, 1)), axis=1).flatten()
                    index += 1
        if index < n:
            temp_preburst = np.delete(temp_preburst, np.s_[index:n], 0)
            temp_nonburst = np.delete(temp_nonburst, np.s_[index:n], 0)
            space_preburst = np.delete(space_preburst, np.s_[index:n], 0)
            space_nonburst = np.delete(space_nonburst, np.s_[index+1:n], 0)

        # combine and shuffle data
        temp_traindata = np.vstack((temp_preburst, temp_nonburst)).astype(int)
        np.random.shuffle(temp_traindata)
        space_traindata = np.vstack(
            (space_preburst, space_nonburst)).astype(int)
        np.random.shuffle(space_traindata)
        # save the result as a csv
        np.savetxt(temp_path + str(w) + '.csv',
                   temp_traindata, delimiter=',')
        np.savetxt(space_path + str(w) + '.csv',
                   space_traindata, delimiter=',')
        print('Total bursts: ' + str(index))

In [5]:
#load an hdf5 file with the simulation data
f = h5py.File(hdf5, 'r')
print(list(f.keys()), '\n')
print(list(f.values()))

['Tsim', 'burstinessHist', 'neuronThresh', 'neuronTypes', 'probedNeurons', 'radiiHistory', 'ratesHistory', 'simulationEndTime', 'spikesHistory', 'spikesProbedNeurons', 'starterNeurons', 'xloc', 'yloc'] 

[<HDF5 dataset "Tsim": shape (1,), type "<f4">, <HDF5 dataset "burstinessHist": shape (60000,), type "<i4">, <HDF5 dataset "neuronThresh": shape (10000,), type "<f4">, <HDF5 dataset "neuronTypes": shape (10000,), type "<i4">, <HDF5 dataset "probedNeurons": shape (10000,), type "<i4">, <HDF5 dataset "radiiHistory": shape (601, 10000), type "<f4">, <HDF5 dataset "ratesHistory": shape (601, 10000), type "<f4">, <HDF5 dataset "simulationEndTime": shape (1,), type "<f4">, <HDF5 dataset "spikesHistory": shape (6000000,), type "<i4">, <HDF5 dataset "spikesProbedNeurons": shape (375898, 10000), type "<u8">, <HDF5 dataset "starterNeurons": shape (1000,), type "<i4">, <HDF5 dataset "xloc": shape (10000,), type "<i4">, <HDF5 dataset "yloc": shape (10000,), type "<i4">]


In [6]:
%%time
# create the spiking data matrix
spikes_loc = create_spike_matrix(f)
# save the spiking data to a csv file
spikes_loc.to_csv(spikes_csv, index=False)

CPU times: user 30min 17s, sys: 1min, total: 31min 18s
Wall time: 29min 47s


In [11]:
# read in burst data
bursts = pd.read_csv(bursts_csv)['StartT'].values
# initialize variables
windows = [5, 10, 50, 100, 500]
gap = 2000
mask = np.in1d(spikes_loc.values[:, 0], bursts)
n = bursts.shape[0]
# get the training data
get_training_data(
    windows, gap, spikes_loc.values[:, 0], spikes_loc.values[:, 1], spikes_loc.values[:, 2], mask, n, temp_path,space_path)

Extracting windows of size 5
Total bursts: 9698
Extracting windows of size 10
Total bursts: 9698
Extracting windows of size 50
Total bursts: 9698
Extracting windows of size 100
Total bursts: 9698
Extracting windows of size 500
Total bursts: 9698

